In [1]:
# edgevit.py

from collections import OrderedDict
import torch
import torch.nn as nn
from functools import partial
import torch.nn.functional as F
import math
from timm.models.vision_transformer import _cfg
from timm.models import register_model
from timm.models.layers import trunc_normal_, DropPath, to_2tuple


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class CMlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Conv2d(in_features, hidden_features, 1)
        self.act = act_layer()
        self.fc2 = nn.Conv2d(hidden_features, out_features, 1)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


import torch
import torch.nn as nn

class SEBlock(nn.Module):
    def __init__(self, dim, reduction=4):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim // reduction)  # Reduce feature size
        self.fc2 = nn.Linear(dim // reduction, dim)  # Restore original size
        self.act = nn.ReLU()  # Non-linearity
        self.sigmoid = nn.Sigmoid()  # Convert values to attention weights

    def forward(self, x):
        """
        Input: x of shape [B, C, H, W]  (Batch, Channels, Height, Width)
        Output: x of shape [B, C, H, W] (Reweighted Feature Map)
        """
        B, C, H, W = x.shape

        # Step 1: Squeeze - Global Average Pooling (GAP)
        w = x.mean(dim=[2, 3])  # Shape: [B, C]

        # Step 2: Excite - Learn importance of channels
        w = self.fc1(w)  # Reduce feature dimension
        w = self.act(w)  # Apply ReLU
        w = self.fc2(w)  # Restore feature dimension
        w = self.sigmoid(w)  # Normalize values to [0,1]

        # Step 3: Scale - Multiply attention weights with original feature map
        w = w.unsqueeze(-1).unsqueeze(-1)  # Reshape to [B, C, 1, 1]
        return x * w  # Element-wise multiplication (channel-wise attention)


class GlobalSparseAttn(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., sr_ratio=1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        # self.upsample = nn.Upsample(scale_factor=sr_ratio, mode='nearest')
        self.sr = sr_ratio
        if self.sr > 1:
            self.sampler = nn.AvgPool2d(1, sr_ratio)
            kernel_size = sr_ratio
            self.LocalProp = nn.ConvTranspose2d(dim, dim, kernel_size, stride=sr_ratio, groups=dim)
            self.norm = nn.LayerNorm(dim)
        else:
            self.sampler = nn.Identity()
            self.upsample = nn.Identity()
            self.norm = nn.Identity()

    def forward(self, x, H: int, W: int):
        B, N, C = x.shape
        if self.sr > 1.:
            x = x.transpose(1, 2).reshape(B, C, H, W)
            x = self.sampler(x)
            x = x.flatten(2).transpose(1, 2)

        qkv = self.qkv(x).reshape(B, -1, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, -1, C)

        if self.sr > 1:
            x = x.permute(0, 2, 1).reshape(B, C, int(H / self.sr), int(W / self.sr))
            x = self.LocalProp(x)
            x = x.reshape(B, C, -1).permute(0, 2, 1)
            x = self.norm(x)

        x = self.proj(x)
        x = self.proj_drop(x)
        return x


# class LocalAgg(nn.Module):
#     def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
#                  drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
#         super().__init__()
#         self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)
#         self.norm1 = nn.BatchNorm2d(dim)
#         self.conv1 = nn.Conv2d(dim, dim, 1)
#         self.conv2 = nn.Conv2d(dim, dim, 1)
#         self.attn = nn.Conv2d(dim, dim, 5, padding=2, groups=dim)
#         self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
#         self.norm2 = nn.BatchNorm2d(dim)
#         mlp_hidden_dim = int(dim * mlp_ratio)
#         self.mlp = CMlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
#
#     def forward(self, x):
#         x = x + self.pos_embed(x)
#         x = x + self.drop_path(self.conv2(self.attn(self.conv1(self.norm1(x)))))
#         x = x + self.drop_path(self.mlp(self.norm2(x)))
#         return x

class LocalAgg(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)  # Positional encoding
        self.attn = nn.Conv2d(dim, dim, 5, padding=2, groups=dim)  # Local attention
        self.norm1 = nn.BatchNorm2d(dim)
        self.norm2 = nn.BatchNorm2d(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = CMlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.se = SEBlock(dim) 

    def forward(self, x):
        identity = x

        # Step 1: Local Feature Extraction
        x = x + self.pos_embed(x)
        x = x + self.attn(self.norm1(x))

        # Step 2: Apply SE Attention (Enhance Important Channels)
        x = self.se(x)

        # Step 3: MLP Block
        x = x + self.mlp(self.norm2(x))
        return x + identity  # Residual connection



class SelfAttn(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1.):
        super().__init__()
        self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)
        self.norm1 = norm_layer(dim)
        self.attn = GlobalSparseAttn(
            dim,
            num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
            attn_drop=attn_drop, proj_drop=drop, sr_ratio=sr_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        # global layer_scale
        # self.ls = layer_scale

    def forward(self, x):
        x = x + self.pos_embed(x)
        B, N, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = x + self.drop_path(self.attn(self.norm1(x), H, W))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        x = x.transpose(1, 2).reshape(B, N, H, W)
        return x


class LGLBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, sr_ratio=1.):
        super().__init__()

        if sr_ratio > 1:
            self.LocalAgg = LocalAgg(dim, num_heads, mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path,
                                     act_layer, norm_layer)
        else:
            self.LocalAgg = nn.Identity()

        self.SelfAttn = SelfAttn(dim, num_heads, mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path, act_layer,
                                 norm_layer, sr_ratio)

    def forward(self, x):
        x = self.LocalAgg(x)
        x = self.SelfAttn(x)
        return x


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.norm = nn.LayerNorm(embed_dim)
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x)
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        x = x.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        return x


class EdgeVit(nn.Module):
    """ Vision Transformer
    A PyTorch impl of : `An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale`  -
        https://arxiv.org/abs/2010.11929
    """

    def __init__(self, depth=[1, 2, 5, 3], img_size=224, in_chans=3, num_classes=1000, embed_dim=[48, 96, 240, 384],
                 head_dim=64, mlp_ratio=4., qkv_bias=True, qk_scale=None, representation_size=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=None, sr_ratios=[4, 2, 2, 1], **kwargs):
        """
        Args:
            depth (list): depth of each stage
            img_size (int, tuple): input image size
            in_chans (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (list): embedding dimension of each stage
            head_dim (int): head dimension
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            norm_layer (nn.Module): normalization layer
        """
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)

        self.patch_embed1 = PatchEmbed(
            img_size=img_size, patch_size=4, in_chans=in_chans, embed_dim=embed_dim[0])
        self.patch_embed2 = PatchEmbed(
            img_size=img_size // 4, patch_size=2, in_chans=embed_dim[0], embed_dim=embed_dim[1])
        self.patch_embed3 = PatchEmbed(
            img_size=img_size // 8, patch_size=2, in_chans=embed_dim[1], embed_dim=embed_dim[2])
        self.patch_embed4 = PatchEmbed(
            img_size=img_size // 16, patch_size=2, in_chans=embed_dim[2], embed_dim=embed_dim[3])

        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depth))]  # stochastic depth decay rule
        num_heads = [dim // head_dim for dim in embed_dim]
        self.blocks1 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[0], num_heads=num_heads[0], mlp_ratio=mlp_ratio[0], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer,
                sr_ratio=sr_ratios[0])
            for i in range(depth[0])])
        self.blocks2 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[1], num_heads=num_heads[1], mlp_ratio=mlp_ratio[1], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0]], norm_layer=norm_layer,
                sr_ratio=sr_ratios[1])
            for i in range(depth[1])])
        self.blocks3 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[2], num_heads=num_heads[2], mlp_ratio=mlp_ratio[2], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0] + depth[1]], norm_layer=norm_layer,
                sr_ratio=sr_ratios[2])
            for i in range(depth[2])])
        self.blocks4 = nn.ModuleList([
            LGLBlock(
                dim=embed_dim[3], num_heads=num_heads[3], mlp_ratio=mlp_ratio[3], qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i + depth[0] + depth[1] + depth[2]],
                norm_layer=norm_layer, sr_ratio=sr_ratios[3])
            for i in range(depth[3])])
        self.norm = nn.BatchNorm2d(embed_dim[-1])

        # Representation layer
        if representation_size:
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(embed_dim, representation_size)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()

        # Classifier head
        self.head = nn.Linear(embed_dim[-1], num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        self.num_classes = num_classes
        self.head = nn.Linear(self.embed_dim, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        x = self.patch_embed1(x)
        x = self.pos_drop(x)
        for blk in self.blocks1:
            x = blk(x)
        x = self.patch_embed2(x)
        for blk in self.blocks2:
            x = blk(x)
        x = self.patch_embed3(x)
        for blk in self.blocks3:
            x = blk(x)
        x = self.patch_embed4(x)
        for blk in self.blocks4:
            x = blk(x)
        x = self.norm(x)
        x = self.pre_logits(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.flatten(2).mean(-1)
        x = self.head(x)
        return x


@register_model
def edgevit_xxs(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 1, 3, 2],
        embed_dim=[36, 72, 144, 288], head_dim=36, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model


@register_model
def edgevit_xs(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 1, 3, 1],
        embed_dim=[48, 96, 240, 384], head_dim=48, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model


@register_model
def edgevit_s(pretrained=True, **kwargs):
    model = EdgeVit(
        depth=[1, 2, 5, 3],
        embed_dim=[48, 96, 240, 384], head_dim=48, mlp_ratio=[4] * 4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), sr_ratios=[4, 2, 2, 1], **kwargs)
    model.default_cfg = _cfg()
    return model
    
@register_model
def edgevit_m(pretrained=False, **kwargs):
    model = EdgeVit(
        depth=[2, 3, 6, 4],  # Increased depth for more layers per stage
        embed_dim=[64, 128, 320, 512],  # Wider embeddings for higher capacity
        head_dim=64,  # Increased head dimension for finer attention
        mlp_ratio=[4, 4, 4.5, 4.5],  # Slightly higher MLP ratio in later stages
        qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        sr_ratios=[4, 2, 2, 1],  # Kept the same, as these are tied to spatial reduction
        **kwargs
    )
    model.default_cfg = _cfg()
    return model


/Users/abhishek/PycharmProjects/EdgeVit_FaceEmo_Faster/.venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [1]:
"""
Real-time Emotion Recognition Webcam Application using EdgeViT_XXS

This script:
1. Captures video from your webcam
2. Detects faces using MediaPipe
3. Preprocesses faces with frequency transforms
4. Performs emotion classification using a pretrained EdgeViT_XXS model
5. Displays results in real-time with visualization
"""

import torch
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import time
import mediapipe as mp

# Emotion labels mapping 
EMOTION_MAP = {
    "angry": 2, "disgust": 1, "fear": 4, "happy": 6,
    "neutral": 3, "sad": 0, "surprise": 5
}

EMOTION_LABELS = {v: k for k, v in EMOTION_MAP.items()}

class EmotionDetector:
    def __init__(self, model_path):
        """Initialize emotion detector with pretrained EdgeViT_XXS model."""
        # Check for GPU
        self.device = torch.device("cpu")
        try:
            self.model = edgevit_xxs(num_classes=7)  
            state_dict = torch.load(model_path, map_location=self.device)
            self.model.load_state_dict(state_dict)  # Load weights
            self.model.to(self.device)
            self.model.eval()  
            print(f"Successfully loaded model from {model_path}")
        except Exception as e:
            print(f"Error loading model: {e}")
        
        # MediaPipe
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=1,  # 1 for full-range detection 
            min_detection_confidence=0.5
        )
        
      
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
    
    def frequency_transform(self, image):
        """Apply frequency transform to create 3-channel representation."""
        # Convert to grayscale if needed
        if len(image.shape) == 3 and image.shape[-1] == 3:  
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Normalize pixel values
        image = image.astype(np.float32) / 255.0  

        # Low-Pass Filter (Gaussian Blur)
        low_pass = cv2.GaussianBlur(image, (7, 7), 1)

        # High-Pass Filter (Sobel Edge Detection)
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        high_pass = cv2.magnitude(sobelx, sobely)
        high_pass = high_pass / np.max(high_pass) if np.max(high_pass) > 0 else high_pass  # Normalize

        # Stack into 3-channel image (Original, Low-Pass, High-Pass)
        freq_image = np.stack([image, low_pass, high_pass], axis=-1)
        return freq_image
    
    def detect_face(self, frame):
        """Detect face in the input frame."""
        
        # Convert to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process frame with face detection
        results = self.face_detection.process(frame_rgb)
        
        # Check if any faces detected
        if not results.detections:
            return None, None
        
        # Get the bounding box of the first detected face
        detection = results.detections[0]  # Using only singl first face
        bboxC = detection.location_data.relative_bounding_box
        
    
        h, w, _ = frame.shape
        
       
        x = max(0, int(bboxC.xmin * w))
        y = max(0, int(bboxC.ymin * h))
        width = min(int(bboxC.width * w), w - x)
        height = min(int(bboxC.height * h), h - y)
        
        # Extract face region
        face_region = frame[y:y+height, x:x+width]
        
        # Return face region and bounding box
        return face_region, (x, y, width, height)
    
    def preprocess_image(self, image):
        """Preprocess image for model inference."""
        # Apply frequency transform
        freq_image = self.frequency_transform(image)
        

        freq_image = (freq_image * 255).astype(np.uint8)
        pil_image = Image.fromarray(freq_image)
        
        # Apply transforms
        tensor_image = self.transform(pil_image)
        
        # Add batch dimension
        tensor_image = tensor_image.unsqueeze(0).to(self.device)
        return tensor_image
    
    def predict_emotion(self, face_image):
        """Predict emotion from face image."""
        # Check if face image is valid
        if face_image is None or face_image.size == 0:
            return None, None
        
        # Preprocess and predict
        with torch.no_grad():
            try:
                tensor_image = self.preprocess_image(face_image)
                outputs = self.model(tensor_image)
                _, predicted = torch.max(outputs, 1)
                
                # Get probabilities
                probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
                
                emotion_idx = predicted.item()
                confidence = probabilities[emotion_idx].item()
                emotion_name = EMOTION_LABELS[emotion_idx]
                
                return emotion_name, probabilities.cpu().numpy()
            except Exception as e:
                print(f"Error during prediction: {e}")
                return None, None



In [ ]:

def run_webcam_app():
    """Run the webcam application for real-time emotion detection."""
  
    model_path = '/Users/abhishek/Downloads/edgevit_finetuned.pth'
    
    # Initialize emotion detector
    detector = EmotionDetector(model_path)
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)  # 0 is usually the default webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    # Setting webcam resolution 
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Variables for FPS calculation
    fps_counter = 0
    fps_start_time = time.time()
    fps = 0
    
    print("Starting webcam emotion detection. Press 'q' to quit.")
    
    # UI settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    line_thickness = 2
    
    while True:
        # Read frame from webcam
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image")
            break
        
        # Create a copy of the frame for display
        display_frame = frame.copy()
        
        # Update FPS calculation
        fps_counter += 1
        if fps_counter >= 10:  # Calculate FPS every 10 frames
            current_time = time.time()
            fps = fps_counter / (current_time - fps_start_time)
            fps_start_time = current_time
            fps_counter = 0
        
        # Detect face
        face_image, face_bbox = detector.detect_face(frame)
        
        # Draw face detection result
        if face_image is not None and face_bbox is not None:
            # Draw bounding box around face
            x, y, w, h = face_bbox
            cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), line_thickness)
            
            # Predict emotion
            emotion, probabilities = detector.predict_emotion(face_image)
            
            if emotion and probabilities is not None:
                # Display emotion above face
                text_y_position = max(y - 10, 20)  # Ensure text doesn't go off-screen
                cv2.putText(display_frame, 
                            f"{emotion.upper()}",
                            (x, text_y_position),
                            font, 0.8, (0, 255, 0), line_thickness)
                
                # Draw emotion probability bars
                bar_width = 150
                bar_height = 15
                bar_gap = 5
                bar_x = 10
                bar_y = 40
                
                # Display emotion probabilities
                
                for i, emotion_name in enumerate(EMOTION_LABELS.values()):
                    # Emotion index
                    emotion_idx = list(EMOTION_LABELS.keys())[i]
                    prob = probabilities[emotion_idx]
                    
                    # Emotion label
                    cv2.putText(display_frame, emotion_name, 
                                (bar_x, bar_y + i*(bar_height+bar_gap+10)), 
                                font, 0.5, (255, 255, 255), 1)
                    
                    # Probability bar background (gray)
                    cv2.rectangle(display_frame, 
                                 (bar_x + 80, bar_y + i*(bar_height+bar_gap+10) - bar_height),
                                 (bar_x + 80 + bar_width, bar_y + i*(bar_height+bar_gap+10)),
                                 (100, 100, 100), -1)
                    
                    # Probability bar (green)
                    bar_length = int(prob * bar_width)
                    cv2.rectangle(display_frame, 
                                 (bar_x + 80, bar_y + i*(bar_height+bar_gap+10) - bar_height),
                                 (bar_x + 80 + bar_length, bar_y + i*(bar_height+bar_gap+10)),
                                 (0, 255, 0), -1)
                    
                    # Display probability percentage
                    cv2.putText(display_frame, f"{prob*100:.1f}%", 
                               (bar_x + 80 + bar_width + 5, bar_y + i*(bar_height+bar_gap+10)),
                               font, 0.5, (255, 255, 255), 1)
                
                # Show the frequency transform of the face
                freq_face = detector.frequency_transform(face_image)
                # Scale to 0-255 for display
                freq_face_display = (freq_face * 255).astype(np.uint8)
                # Resize for better display
                freq_face_display = cv2.resize(freq_face_display, (224, 224))
                cv2.imshow("Frequency Transform", freq_face_display)
        else:
            # No face detected
            cv2.putText(display_frame, "No face detected", (10, 30), 
                        font, 0.8, (0, 0, 255), line_thickness)
        
        # Display FPS
        cv2.putText(display_frame, f"FPS: {int(fps)}", 
                   (display_frame.shape[1]-120, 30), 
                   font, 0.7, (0, 0, 255), 2)
        
        # Display the result
        cv2.imshow("Emotion Recognition", display_frame)
        
        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Application closed.")


In [ ]:
run_webcam_app()